# Twitter GeoTrend Monitor

This notebook uses Twitter's API to search the trending hashtags and keywords in the top 50 most populous cities of India and records them in  __[this Google Sheet](http://https://docs.google.com/spreadsheets/d/1UR2LSzosYvXARe2vwM4E0c4zZXZMgGuhG1xhgnq0pyc/edit#gid=723932942)__. It also caculates the top 20 most common trends across India.

You should have the files `keys.json` containing your Twitter API keys  and `pygsheets_authfile.json` containing your Google Sheets credentials saved in the same folder as this notebook in order to access Twitter's API and Google Sheets.


In [1]:
#import the necessary packages
import sys
import json
import pandas as pd
import numpy as np
import datetime as dt
import pprint as pp
import ast
import collections

try:
    import tweepy
except:
    !{sys.executable} -m pip install tweepy
    import tweepy
     
try:
    import pygsheets
except:
    !{sys.executable} -m pip install pygsheets
    import pygsheets

#log into google sheets
gc = pygsheets.authorize(service_file='pygsheets_authfile.json')
    
# log into twitter 
key_file = 'keys.json'
with open(key_file) as f:
    keys = json.load(f)

try:
    auth = tweepy.OAuthHandler(keys["consumer_key"], keys["consumer_secret"])
    auth.set_access_token(keys["access_token"], keys["access_token_secret"])
    api = tweepy.API(auth)
    print("Your username is:", api.auth.get_username())
except TweepError as e:
    logging.warning("There was a Tweepy error. Double check your API keys and try again.")
    logging.warning(e)

Your username is: s_onnet


### Get lat/longs from google sheet

In [9]:
sheet_url = 'https://docs.google.com/spreadsheets/d/1UR2LSzosYvXARe2vwM4E0c4zZXZMgGuhG1xhgnq0pyc/edit#gid=0'
lat_long_df = get_df_from_sheet(sheet_url, 0)
lat_long_df.head()

,City (by size population),Latitude,Longitude
0,Mumbai,19.07283,72.88261
1,New Delhi,28.65381,77.22897
2,Bangalore,12.97194,77.59369
3,Hyderabad,17.38405,78.45636
4,Ahmedabad,23.02579,72.58727
5,Chennai,13.08784,80.27847
6,Kolkata,22.56263,88.36304
7,Surat,21.19594,72.83023
8,Pune,18.51957,73.85535
9,Jaipur,26.91962,75.78781


In [2]:
def get_df_from_sheet(url, page_num):
    sheet = gc.open_by_url(url)
    return pd.DataFrame(sheet.worksheet(page_num).get_all_records())

def search_all_coords(df):
    '''Adds column to dataframe with trends'''
    gen = df.iterrows()
    df['Trends'] = np.nan
    for i in range(len(df)):
        city = next(gen)
        try:
            trends = get_trends(city[1]['Latitude'], city[1]['Longitude'])
            df['Trends'][i] = clean_trend(trends)
        except TweepError: 
            print('You hit the rate limit!)
            break
    return df

def get_trends(lat, long):
    '''Takes in latitude and longitude of site/city of interest
    Returns array of json-type Trend objects
    '''
    closest_places = api.trends_closest(lat, long)
    if len(closest_places) == 0:
        return "No nearby trends."
    trends = {}
    for place in closest_places:
        trends[f'{place}'] = api.trends_place(place['woeid'])
   
    #print(f"The top trends in {place['woeid']} right now are:" + str(list(trends_df['name'])))
    return trends

def clean_trend(trend):
    '''Isolates trend name from Trend object'''
    trends_df = pd.DataFrame.from_dict(trend[list(trend.keys())[0]][0]['trends'])
    return list(trends_df['name'])

def df_to_gsheet(df, title, sheet_url, gc):
    '''Writes dataframe into a new worksheet in a given Google Sheet'''
    sh = gc.open_by_url(sheet_url)
    wks = sh.add_worksheet(f'{title} on {dt.datetime.now().strftime("%Y-%m-%d %H:%M")}')
    wks.set_dataframe(df, (1,1))
    return sh
                           
def count_frequent_trends(df, n):
    '''
    Takes in a list of words
    Returns a dataframe with their individual frequencies
    ''' 
    trends = []
    for i in df['Trends']:
        trends += i 
    print(trends)
    trend_count = pd.DataFrame(collections.Counter(trends).most_common(), columns=['Word', 'Frequency'])
    print(f"The {n} most common trending topics across India right now are: " + str(list(trend_count['Word'].iloc[0:n])))
    pp.pprint(trend_count.head(n))
    return trend_count


In [26]:
#DON'T RUN OFTEN TO AVOID BEING RATE LIMITED
#trends = search_all_coords(lat_long_df) #Only run when ready to call Twitter API
trends_df = pd.concat([trends, pd.DataFrame(trends['Trends'].values.tolist())], axis=1).drop(columns=['Trends'])
df_to_gsheet(trends_df, 'Trends', sheet_url, gc)
df_to_gsheet(count_frequent_trends(trends, 20), 'Trend Frequencies', sheet_url, gc)
trends_df.head()

/Users/sonnet/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/sonnet/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


['#LIVBAR', '#DSSAidsThalassemiaPatients', 'Liverpool', '#WednesdayWisdom', '#YNWA', '#WednesdayMotivation', 'Barcelona', 'Barca', 'Anfield', 'Colorado', 'Valverde', 'Jurgen Klopp', 'Roma', 'Google Pixel 3a', 'Denver', 'Istanbul', 'Essar Steel', 'Reds', 'Origi', 'robertson', 'No Salah', 'Rajiv Gandhi', 'Qualcomm Snapdragon 855', '#ChampionsLeague', '#MaharshiFromTomorrow', '#LFCBarça', '#SDLive', '#WorldThalassemiaDay', '#LIVFCB', '#UCLfinal', '#Klopp', '#Pixel3aXL', '#NeverGiveUp', '#FANBOOST', '#passioneering', '#MonacoEPrix', '#LFCFCB', '#Hippi', '#MIvCSK', '#CSKvMI', '#MetGala', '#io19', '#AkshayaTritiya', '#RabindranathTagore', '#GoogleIO2019', '#HarDilMeiModi', '#RamadanMubarak', '#IPLPlayoffs', '#TuesdayThoughts', '#ParshuramJayanti', 'Rajiv Gandhi', 'Ramadan', 'श्री राम', '#GameofThrones', 'राहुल गांधी', 'Madam', 'Muslims', 'Jai Shree Ram', '#LIVBAR', '#DSSAidsThalassemiaPatients', 'Liverpool', '#WednesdayWisdom', '#YNWA', 'Barcelona', 'Barca', 'Anfield', 'Colorado', 'Valverde'

<Spreadsheet 'Geotrend Tracker' Sheets:6>